In [6]:
import re


TOKENS = {
    'ID': r'[a-zA-Z_][a-zA-Z0-9_]*',
    'NUM': r'\d+',
    'ASSIGN': r'=',
    'PLUS': r'\+',
    'LBRACK': r'\[',
    'RBRACK': r'\]',
    'SEMICOLON': r';'
}

class Lexer:
    def __init__(self, source_code):
        self.source_code = source_code
        self.tokens = []
        self.tokenize()

    def tokenize(self):
        pattern = '|'.join(f'(?P<{key}>{value})' for key, value in TOKENS.items())
        for match in re.finditer(pattern, self.source_code):
            for token_name, token_value in match.groupdict().items():
                if token_value:
                    self.tokens.append((token_name, token_value))

    def get_tokens(self):
        return self.tokens


class SDTSParser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.index = 0
        self.temp_count = 0
        self.tac_code = []

    def new_temp(self):
        """Generate new temporary variable"""
        self.temp_count += 1
        return f"t{self.temp_count}"

    def match(self, expected_type):
        """Match and consume the expected token"""
        if self.index < len(self.tokens) and self.tokens[self.index][0] == expected_type:
            self.index += 1
        else:
            raise SyntaxError(f"Expected {expected_type}, got {self.tokens[self.index][0]}")

    def parse_array_index(self):
        """Parse an array index like [i] and return its variable"""
        self.match('LBRACK')
        index_var = self.tokens[self.index][1]
        self.match('ID')
        self.match('RBRACK')
        return index_var

    def parse_assignment(self):
        """Parse 3D array assignment and generate Three Address Code"""
        base_array = self.tokens[self.index][1]
        self.match('ID')

        # Parsing [i][j][k]
        i_temp = self.parse_array_index()
        j_temp = self.parse_array_index()
        k_temp = self.parse_array_index()

        self.match('ASSIGN')
        value_temp = self.tokens[self.index][1]
        self.match('ID')
        self.match('SEMICOLON')


        D2, D3, size = 10, 10, 4

        t1 = self.new_temp()
        self.tac_code.append(f"{t1} = {i_temp} * {D2 * D3}")

        t2 = self.new_temp()
        self.tac_code.append(f"{t2} = {j_temp} * {D3}")

        t3 = self.new_temp()
        self.tac_code.append(f"{t3} = {t1} + {t2}")

        t4 = self.new_temp()
        self.tac_code.append(f"{t4} = {t3} + {k_temp}")

        t5 = self.new_temp()
        self.tac_code.append(f"{t5} = {t4} * {size}")

        address = self.new_temp()
        self.tac_code.append(f"{address} = &{base_array} + {t5}")


        self.tac_code.append(f"MEM[{address}] = {value_temp}")

    def generate_TAC(self):
        """Return the generated Three Address Code (TAC)"""
        return self.tac_code


def format_TAC(tac_code):
    print("\nThree Address Code (TAC) Generated:\n")
    for line in tac_code:
        print(line)


if __name__ == "__main__":

    code = "A[i][j][k] = B;"

    lexer = Lexer(code)
    tokens = lexer.get_tokens()

    parser = SDTSParser(tokens)
    parser.parse_assignment()

    format_TAC(parser.generate_TAC())



Three Address Code (TAC) Generated:

t1 = i * 100
t2 = j * 10
t3 = t1 + t2
t4 = t3 + k
t5 = t4 * 4
t6 = &A + t5
MEM[t6] = B
